<a href="https://colab.research.google.com/github/AyushSoni14/Deep-Learning-for-Musculoskeletal-Dynamics-and-Transformation-Approximation/blob/main/Arm_motion_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

data = pd.read_csv('/content/filtered_output (3).csv')
data = data.values

input_dim = 28
length_dim = 34
force_dim = 165
torque_dim = 123

X = data[:, :input_dim]
Y_length = data[:, input_dim:input_dim+length_dim]
Y_force = data[:, input_dim+length_dim:input_dim+length_dim+force_dim]
Y_torque = data[:, input_dim+length_dim+force_dim:]

X_temp, X_test, Y_length_temp, Y_length_test, Y_force_temp, Y_force_test, Y_torque_temp, Y_torque_test = train_test_split(
    X, Y_length, Y_force, Y_torque, test_size=0.05, random_state=42
)
X_train, X_val, Y_length_train, Y_length_val, Y_force_train, Y_force_val, Y_torque_train, Y_torque_val = train_test_split(
    X_temp, Y_length_temp, Y_force_temp, Y_torque_temp, test_size=0.2, random_state=42
)

print("Data splits:")
print("Training samples:", X_train.shape[0])
print("Validation samples:", X_val.shape[0])
print("Test samples:", X_test.shape[0])

def build_model(output_dim, layer_sizes):
    model = models.Sequential([layers.Input(shape=(input_dim,))] +
                              [layers.Dense(size, activation='relu') for size in layer_sizes] +
                              [layers.Dense(output_dim, activation='linear')])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mse',
                  metrics=['mae'])
    return model

length_model = build_model(length_dim, [1024, 512])
force_model = build_model(force_dim, [2048, 1024])
torque_model = build_model(torque_dim, [2048, 1024])

es = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

epochs = 50
batch_size = 256

models_and_data = [(length_model, Y_length_train, Y_length_val, "Muscle Length"),
                   (force_model, Y_force_train, Y_force_val, "Force"),
                   (torque_model, Y_torque_train, Y_torque_val, "Torque")]

histories = {}
for model, Y_train, Y_val, name in models_and_data:
    print(f"\nTraining {name} model...")
    history = model.fit(X_train, Y_train,
                        validation_data=(X_val, Y_val),
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[es],
                        verbose=1)
    histories[name] = history

results = {}
for model, Y_test, name in [(length_model, Y_length_test, "Muscle Length"),
                            (force_model, Y_force_test, "Force"),
                            (torque_model, Y_torque_test, "Torque")]:
    loss, mae = model.evaluate(X_test, Y_test, verbose=0)
    results[name] = (loss, mae)
    print(f"\n{name} Model - Loss (MSE): {loss:.6f}, MAE: {mae:.6f}")

plt.figure(figsize=(12, 5))
for i, (name, history) in enumerate(histories.items()):
    plt.subplot(1, 3, i+1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title(f'{name} Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('MSE Loss')
    plt.legend()
plt.tight_layout()
plt.show()

for model, Y_test, name in [(length_model, Y_length_test, "Muscle Length"),
                            (force_model, Y_force_test, "Force"),
                            (torque_model, Y_torque_test, "Torque")]:
    Y_pred = model.predict(X_test)
    rmse = np.sqrt(np.mean((Y_test - Y_pred)**2, axis=0))
    plt.hist(rmse, bins=20, alpha=0.7, label=name)

plt.title('Histogram of RMSE for All Models')
plt.xlabel('RMSE')
plt.ylabel('Frequency')
plt.legend()
plt.show()
